# Ecosystem Services Valuation Workflow

This notebook follows the 11-phase roadmap from the project README. Fill in each section sequentially to maintain provenance and reproducibility.

## 0. Setup
- Activate the `biomass` environment (`conda activate biomass`).
- Update the project path, AOI coordinates, and CDSE credentials in the cells below.

In [1]:
from pathlib import Path
import json

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)
print(f"Data directory: {DATA_DIR.resolve()}")

Data directory: /home/fred/Projects/Feishu-Hack/Biomass_fun/data


In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

print("Core libraries imported.")

Core libraries imported.


## Phase 0: CDSE API & Download
- Implement a reusable CDSE authentication helper (env vars or config file).
- Load AOI centroid coordinates and build a 25 km buffer polygon (`geopandas`/`shapely`).
- Use the CDSE search endpoint to query Sentinel-2 L2A products intersecting the AOI.
- Filter candidates by acquisition date, cloud cover, and processing baseline.
- Log the chosen product(s) with metadata (UUID, title, footprint, cloud %, download URL).
- Download the `.SAFE` archives into `DATA_DIR` and verify checksums if provided.

In [3]:
from datetime import datetime
import pandas as pd

SAFE_ROOTS = sorted(Path.cwd().glob("S2C_MSIL2A_*.SAFE"))
if not SAFE_ROOTS:
    raise FileNotFoundError("No Sentinel-2 .SAFE directories were found in the project root.")


def summarize_safe(path: Path) -> dict:
    tokens = path.name.split("_")
    return {
        "safe_name": path.name,
        "acquisition_utc": pd.to_datetime(tokens[2], format="%Y%m%dT%H%M%S"),
        "processing_baseline": tokens[3],
        "relative_orbit": tokens[4],
        "tile": tokens[5],
        "local_path": path.resolve().as_posix(),
    }

product_log = pd.DataFrame([summarize_safe(p) for p in SAFE_ROOTS])
product_log


,safe_name,acquisition_utc,processing_baseline,relative_orbit,tile,local_path
0,S2C_MSIL2A_20251113T024511_N0511_R089_T50RQV_2...,2025-11-13 02:45:11,N0511,R089,T50RQV,/home/fred/Projects/Feishu-Hack/Biomass_fun/S2...
1,S2C_MSIL2A_20251113T024511_N0511_R089_T51RTQ_2...,2025-11-13 02:45:11,N0511,R089,T51RTQ,/home/fred/Projects/Feishu-Hack/Biomass_fun/S2...


## Phase 1: Data Loading & Preprocessing
- Parse the `.SAFE` structure to locate band JP2 paths (10 m & 20 m resolutions).
- Load B2, B3, B4, B8, B11, B12 arrays; resample 20 m bands to the 10 m reference grid.
- Apply the scaling factor (÷10,000) and cast to `float32` reflectance.
- Generate nodata masks (<=0) and cloud masks using `MSK_CLDPRB_20m` or SCL rasters.
- Combine masks into a single `valid` mask to keep clean pixels for downstream phases.
- Build quick-look products: RGB, SWIR composites, cloud overlays for QA/QC.

## Phase 2: Vegetation Indices
- Implement NDVI, EVI, and optional SAVI with epsilon protection on denominators.
- Clip outputs to the [-1, 1] range and set invalid pixels to `NaN` using `MASKS`.
- Summarize each index with descriptive stats (min, max, mean, median, std, percentiles).
- Plot map panels and histograms to confirm distributions and detect anomalies.
- Flag out-of-range values before feeding indices into classification or biomass models.

## Phase 3: Land Cover Classification
- Define NDVI-based thresholds for `non-forest`, `sparse`, `moderate`, and `dense` classes.
- Create a categorical raster using the thresholds and `MASKS["valid"]`.
- Calculate area (ha) per class using pixel area conversions.
- Plot a classification map with an intuitive color palette.
- Summarize counts/areas in a table for reporting.


## Phase 4: Biomass Estimation
- Select or cite an allometric equation suitable for the study region.
- Apply the equation pixel-wise to generate a biomass raster (tons/ha).
- Mask low-vegetation pixels (e.g., NDVI < 0.2) before aggregating.
- Enforce realistic min/max biomass bounds to avoid outliers.
- Compute total biomass (tons) using pixel area (0.01 ha at 10 m).
- Visualize biomass distribution with a legend and summary statistics.


## Phase 5: Analysis & Validation
- Derive descriptive stats (mean, median, std, min, max) for biomass and key indices.
- Build histograms/boxplots to inspect distributions and potential skew.
- Compare aggregated values to literature benchmarks for similar ecosystems.
- Conduct sensitivity checks on main parameters (e.g., NDVI thresholds, coefficients).
- Document assumptions, uncertainties, and data quality caveats.


## Phase 6: Water Detection
- Compute NDWI and MNDWI using the reflectance bands already loaded.
- Apply tuned thresholds to delineate water bodies.
- Separate water classes by permanence/intensity if possible.
- Calculate total water area and percentage of valid pixels.
- Map the water mask(s) with transparent overlays on RGB for QA.


## Phase 7: Water Quality
- Calculate a turbidity proxy such as the Normalized Difference Turbidity Index (NDTI).
- Restrict calculations to water pixels only.
- Create quality classes (e.g., healthy, moderate, degraded).
- Plot maps showing turbidity gradients.
- Note potential confounders (suspended sediment, sensor noise).


## Phase 8: Hydrological Analysis
- Build riparian buffer zones (30 m, 100 m, 300 m) using AOI geometry.
- Summarize NDVI or biomass statistics within each buffer to assess vegetation quality.
- Evaluate wetland connectivity metrics (distance to water, corridor quality, area).
- Flag priority conservation corridors based on connectivity + condition.
- Document formulas/assumptions for reproducibility.


## Phase 9: Ecosystem Service Quantification
- Translate biophysical metrics to per-pixel service scores for the five services.
- **Water Flow Regulation:** estimate water storage capacity using NDVI-based vegetation factors.
- **Water Purification:** compute pollutant removal capacity using NDVI and wetland area fractions.
- **Sediment Control:** approximate sediment retention via simplified USLE proxies (NDVI, buffers).
- **Aquifer Recharge:** estimate recharge potential with precipitation + NDVI-derived infiltration.
- **Flood Protection:** approximate flood storage using floodplain extent, storage depth, roughness.
- Produce maps and tables for each service, highlight hotspots.


## Phase 10: Dynamic Ecosystem Service Valuation
- Assign base value coefficients ($/ha/year) to each service (per README guidance).
- Derive Quality, Scarcity, and Benefit multipliers from indices and contextual data.
- Compute per-pixel dynamic value = base × quality × scarcity × benefit.
- Aggregate to total annual value and unit value (per ha) for the AOI.
- Generate valuation maps and concise summary tables.


## Phase 11: Documentation & Presentation
- Write a clear methodology narrative covering input data, preprocessing, models, and assumptions.
- Export publication-ready figures (maps, histograms, tables) with consistent styling.
- Compile a concise PDF report (3–5 pages) summarizing objectives, methods, results, discussion.
- Prepare a short slide deck (≈5 minutes) highlighting key insights and visuals.
- Ensure notebook cells are clean, commented, and reproducible for final submission.
